# Desafio 1

link a los datos: http://campus.digitalhouse.com/mod/url/view.php?id=32168
Es necesario guardar el csv con el nombre properatti.csv en una carpeta DATA en el raiz del proyecto

In [1]:
#Importar librerias
import numpy as np
import pandas as pd
import re
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Leemos el archivo y creamos el dataframe "data"
data = pd.read_csv("./DATA/properatti.csv")
data.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [3]:
#1. #Renombramos Columnas
data.rename(columns={'Unnamed: 0':'ID',
                    'property_type':'TIPO_PROPIEDAD'}, inplace = True) 

In [4]:
#2. Agregamos 4 columnas al dataframe "data" (PAIS/PROVINCIA/CIUDAD/BARRIO) a partir de place_with_parent_names,place_name.

#2.1 Spliteamos para sacar el pipe y creamos un nuevo dataframe (df_ubicacion) que luego joinearemos para pegarle las 4 columnas

pais_lista =  [elemento[1] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
provincia_region_lista =  [elemento[2] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
barrio_ciudad =  [elemento[3] for elemento in [sublista.split('|') for sublista in data.place_with_parent_names]]
barrio  = data['place_name'].tolist()

d = {'PAIS':pais_lista ,
     'PROVINCIA':  provincia_region_lista  ,
     'CIUDAD':barrio_ciudad,
     'BARRIO':barrio,
     'ID': data['ID'].tolist()
    }
df_ubicacion = pd.DataFrame(data=d)

#Si PROVINCIA = CABA -> a ciudad asignar CABA.
df_ubicacion['CIUDAD'] = np.where(df_ubicacion['PROVINCIA'] =='Capital Federal','Capital Federal',df_ubicacion['CIUDAD'])

#Si CIUDAD = BARRIO -> BARRIO DEJARLO EN NAN
df_ubicacion['BARRIO'] = np.where(df_ubicacion['BARRIO'] ==df_ubicacion['CIUDAD'],'SIN DATO',df_ubicacion['BARRIO'])

#Si CIUDAD esta vacio ('') y barrio == a pronvincia , llenar CIUDAD Y BARRIO CON NAN
df_ubicacion['BARRIO'] = np.where((df_ubicacion['BARRIO'] ==df_ubicacion['PROVINCIA']) & (df_ubicacion['CIUDAD'] =='') ,'SIN DATO',df_ubicacion['BARRIO'])
df_ubicacion['CIUDAD'] = np.where((df_ubicacion['BARRIO'] ==df_ubicacion['PROVINCIA']) & (df_ubicacion['CIUDAD'] =='') ,'SIN DATO',df_ubicacion['CIUDAD'])
df_ubicacion['CIUDAD'] = np.where(df_ubicacion['CIUDAD'] =='', 'SIN DATO',df_ubicacion['CIUDAD'])

#2.2agregamos esta informacion al data frame DATA
data = pd.merge(data, df_ubicacion, on='ID', how='left')
data.rename(columns={'CIUDAD':'CIUDAD-PARTIDO'}, inplace=True)



In [5]:
# Extraer los m2 y completar faltantes - Santi

In [6]:
# Extraer los ambientes y completar faltantes - Fer

In [7]:
# Regex para determinar ambientes

In [8]:
patron = r'(\d*|mono|\w*)\s*?amb\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["title_dato"] = data.title.str.extract(r'(\d*|mono|\w*)\s*?amb\w*', expand=False)

In [9]:
patron = r'(\d*|MONO|\w*)\s*?AMB\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["title_dato1"] = data.title.str.extract(r'(\d*|MONO|\w*)\s*?AMB\w*', expand=False)

In [10]:
patron = r'(\d*|mono|\w*)\s*?amb\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["description_dato"] = data.description.str.extract(r'(\d*|mono|\w*)\s*?amb\w*', expand=False)

In [11]:
patron = r'(\d*|MONO|\w*)\s*?AMB\w*'
regex = re.compile(patron, flags=re.IGNORECASE)
data["description_dato1"] = data.description.str.extract(r'(\d*|MONO|\w*)\s*?AMB\w*', expand=False)

In [12]:
# Reemplazar los valores NaN por 0

In [13]:
data["title_dato"]= data["title_dato"].fillna(0)
data["title_dato1"]= data["title_dato1"].fillna(0)
data["description_dato"]= data["description_dato"].fillna(0)
data["description_dato1"]= data["description_dato1"].fillna(0)

In [14]:
# Generar datos de ambientes con palabras clave

In [15]:
data.replace({'title_dato': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO":1, "mono":1, "DOS":2, "dos":2, "TRES":3, "tres":3, "CUATRO":4, "cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'title_dato1': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'description_dato': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)
data.replace({'description_dato1': {'4º':0,'3º':0,'½':0,"Mono":1,"siete":7,"IMPORTANTE":1,"importante":1,"0002":2,"AMPLISIMO":1,"hermoso":1,"9":9,"8":8,"7":7,"6":6,"5":5,"4":4,"3":3,"2":2,"1":1,"0":0,"Comodo":1,"amplio":1,"Unico":1,"gran":1,"Único":1,"Amplio":1,"un":1,"MONOH":1,"MON":1,"AMPLIO":1,"BMONO":1,"M0NO":1,"UN":1,"MONO": 1, "mono": 1, "DOS" : 2, "dos": 2, "TRES": 3,"tres":3,"CUATRO":4,"cuatro":4,"CINCO":5, "cinco":5}},  inplace = True)

In [16]:
# Los valores texto indetrminados los convierto en 0

In [17]:
data["title_dato"]=data["title_dato"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["title_dato1"]=data["title_dato1"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["description_dato"]=data["description_dato"].replace(r'[a-z]|[A-Z]', 0, regex=True)
data["description_dato1"]=data["description_dato1"].replace(r'[a-z]|[A-Z]', 0, regex=True)

In [18]:
#convierto los datos a numeric para poder calcular los ambientes

In [19]:
data[['title_dato','title_dato1','description_dato','description_dato1']] = data[['title_dato','title_dato1','description_dato','description_dato1']].apply(pd.to_numeric)

In [20]:
#Calculo los ambientes

In [21]:
data['Ambientes']=data[['title_dato','title_dato1','description_dato','description_dato1']].apply(np.max,axis=1)

In [22]:
#Convierto Ambientes a int 

In [23]:
data['Ambientes']=data['Ambientes'].astype(int)

In [24]:
#elimino las columnas de calculo

In [25]:
data = data.drop('title_dato', 1)
data = data.drop('title_dato1', 1)
data = data.drop('description_dato', 1)
data = data.drop('description_dato1', 1)

In [26]:
#FIN AMBIENTES

In [27]:
# Detectar dptos a pozo para eliminar del análisis - Juan
patron = r'(pozo|cuota|proyec|emprendim)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["pozo"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

# Y con esto los elimino
data = data[data.pozo == 0]
data = data.drop(['pozo'], axis = 1)

In [28]:
# Agregar dummies de características que puedan influenciar (ammenities, cochera, etc.) - Juan
patron = r'(pileta|pisci)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["pileta"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(ameni|gimn|gym|laundry|sum|usos mult|solarium)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["amenities"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(estre)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["a estrenar"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(subte)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["subte"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

patron = r'(garage|cocher)'
regex = re.compile(patron, flags=re.IGNORECASE)
data["cochera"] = 1 * data.description.apply(lambda x: bool(regex.search(str(x))))

# Contar casos de cada una de las características

data.loc[:,'pileta':'cochera'].sum()

pileta        25811
amenities     18293
a estrenar     9323
subte          5640
cochera       40946
dtype: int64

In [29]:
# Agregar variables geográficas (distancias a subtes, comisarías, escuelas, etc.) - Santi (sólo si hay tiempo)

In [30]:
# Crear nuevas columnas de precio x m2 y x m2 cubierto a partir de la nueva info - Juan
# Pongo la cuenta con las varaibles actuales, cuando creemos los nuevos campos hay que cambiarles el nombre
data["USD_por_m2_total"] = data['price_aprox_usd'] / data['surface_total_in_m2']
data["USD_por_m2_cubierto"] = data['price_aprox_usd'] / data['surface_covered_in_m2']

In [31]:
# Algún tratamiento de outlyers?

#Devuelve como queda distribuida la base mirando PRECIO EN DOLARES
#data.price_aprox_usd.describe().astype(int)
#Devuelve como queda distribuida la base mirando PRECIO EN DOLARES tomando 2 desvidos standard
#data[np.abs(data.price_aprox_usd-data.price_aprox_usd.mean()) <= (2*data.price_aprox_usd.std())].price_aprox_usd.describe().astype(int)
#Filtrar a aquellos que quedan mas alla de 2 desvios standard. OJO QUE AL EJECUTAR PISA EL DATAFRAME
data = data[np.abs(data.price_aprox_usd-data.price_aprox_usd.mean()) <= (3*data.price_aprox_usd.std())]

In [32]:
# Borramos columnas que ya no necesitamos 
del data['place_name']
del data['place_with_parent_names']
del data['country_name']
del data['state_name']
del data['geonames_id']
del data['operation']


data.head()

,ID,TIPO_PROPIEDAD,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,...,PAIS,PROVINCIA,Ambientes,pileta,amenities,a estrenar,subte,cochera,USD_por_m2_total,USD_por_m2_cubierto
0,0,PH,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,...,Argentina,Capital Federal,2,0,0,0,0,0,1127.272727,1550.000000
1,1,apartment,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,...,Argentina,Bs.As. G.B.A. Zona Sur,0,0,0,1,0,1,NaN,NaN
2,2,apartment,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,...,Argentina,Capital Federal,2,0,0,0,0,0,1309.090909,1309.090909
3,3,PH,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,...,Argentina,Capital Federal,3,0,0,0,0,0,NaN,NaN
4,4,apartment,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,...,Argentina,Buenos Aires Costa Atlántica,2,0,0,0,0,0,1828.571429,1828.571429


In [44]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


data['surface_total_in_m2'] = np.where(data['surface_total_in_m2'].isnull() | data['surface_total_in_m2'] == 0 ,data['surface_covered_in_m2'],data['surface_total_in_m2'])
data['price_usd_per_m2'] = np.where((data['price_usd_per_m2'].isnull() | data['price_usd_per_m2'] == 0) & (data['surface_total_in_m2'] > 0 & data['surface_total_in_m2'].notnull()),data['price_aprox_usd'] / data['surface_total_in_m2'], data['price_usd_per_m2'])
data_final = data.dropna(subset=['price_aprox_usd', 'surface_total_in_m2'])
data_final.head()

,ID,TIPO_PROPIEDAD,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,BARRIO,CIUDAD-PARTIDO,PAIS,PROVINCIA,Ambientes,pileta,amenities,a estrenar,subte,cochera,USD_por_m2_total,USD_por_m2_cubierto
0,0,PH,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Mataderos,Capital Federal,Argentina,Capital Federal,2,0,0,0,0,0,1127.272727,1550.000000
2,2,apartment,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Mataderos,Capital Federal,Argentina,Capital Federal,2,0,0,0,0,0,1309.090909,1309.090909
4,4,apartment,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Centro,Mar del Plata,Argentina,Buenos Aires Costa Atlántica,2,0,0,0,0,0,1828.571429,1828.571429
6,6,PH,"-34.5329567,-58.5217825",-34.532957,-58.521782,130000.0,USD,2293785.0,130000.0,106.0,78.0,1226.415094,1666.666667,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,Munro,Vicente López,Argentina,Bs.As. G.B.A. Zona Norte,0,0,0,0,0,0,1226.415094,1666.666667
7,7,apartment,"-34.5598729,-58.443362",-34.559873,-58.443362,138000.0,USD,2434941.0,138000.0,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,Belgrano,Capital Federal,Argentina,Capital Federal,1,1,1,1,0,0,3066.666667,3450.000000


In [37]:
#Crear y guardar la info en un nuevo csv
data_final.to_csv("./DATA/properattilimpio.csv")